# Stock Validation - Automated Email Notification

**Purpose:** Automated email reporting for stock data validation between BIPROD and Azure

**Author:** Mehmet Cetin  
**GitHub:** https://github.com/mcetin-data

---

## Overview

This notebook connects to the Power BI Stock Validation dataset, executes DAX queries to retrieve KPIs, formats a professional HTML email, and sends it to stakeholders via Microsoft Graph API.

**Scheduled to run:** Twice per week (Monday & Thursday, 8:00 AM)

---

## Step 1: Setup & Imports

In [ ]:
# Import required libraries
import sempy.fabric as fabric
import pandas as pd
from datetime import datetime, timedelta
import requests
from notebookutils import mssparkutils

print("✅ Libraries imported successfully")
print(f"📅 Execution Date: {datetime.now().strftime('%d/%m/%Y %H:%M')}")

## Step 2: Connect to Power BI Dataset

In [ ]:
# Dataset configuration
WORKSPACE_NAME = "Your_Workspace_Name"  # Replace with your workspace
DATASET_NAME = "Stock_Validation"       # Replace with your dataset name

print(f"🔗 Connecting to dataset: {DATASET_NAME}")
print(f"📂 Workspace: {WORKSPACE_NAME}")

## Step 3: Execute DAX Queries to Get KPIs

In [ ]:
# DAX query to retrieve all key metrics
dax_query = """
EVALUATE
ROW(
    "Azure_Records", [Azure_Records],
    "BIPROD_Records", [BIPROD_Records],
    "Records_Diff", [Records_Diff],
    
    "Azure_Unique_Barcodes", [Azure_Unique_Barcodes],
    "BIPROD_Unique_Barcodes", [BIPROD_Unique_Barcodes],
    "Barcodes_Diff", [Barcodes_Diff],
    
    "Azure_Total_QTY", [Azure_Total_QTY],
    "BIPROD_Total_QTY", [BIPROD_Total_QTY],
    "Total_QTY_Diff_Correct", [Total_QTY_Diff_Correct],
    "QTY_Diff_Pct", [QTY_Diff_Pct],
    
    "Azure_Total_COGS", [Azure_Total_COGS],
    "BIPROD_Total_COGS", [BIPROD_Total_COGS],
    "Total_COGS_Diff_Correct", [Total_COGS_Diff_Correct],
    "COGS_Diff_Pct", [COGS_Diff_Pct],
    
    "Match_Count", [Match_Count],
    "Mismatch_Count", [Mismatch_Count],
    "Azure_Only_Count", [Azure_Only_Count],
    "BIPROD_Only_Count", [BIPROD_Only_Count],
    "Alignment_Pct", [Alignment_Pct],
    "Health_Status", [Health_Status]
)
"""

# Execute DAX query
try:
    results = fabric.evaluate_dax(
        dataset=DATASET_NAME,
        workspace=WORKSPACE_NAME,
        dax_string=dax_query
    )
    
    # Convert to DataFrame for easier handling
    df = pd.DataFrame(results)
    
    print("✅ DAX query executed successfully")
    print(f"📊 Retrieved {len(df.columns)} metrics")
    
except Exception as e:
    print(f"❌ Error executing DAX query: {str(e)}")
    raise

## Step 4: Extract KPI Values

In [ ]:
# Extract values from DataFrame (first row)
kpis = df.iloc[0]

# Format date
report_date = datetime.now().strftime("%d/%m/%Y")
next_report = (datetime.now() + timedelta(days=3)).strftime("%d/%m/%Y")

# Determine status color based on alignment
alignment_pct = kpis['[Alignment_Pct]']
if alignment_pct >= 0.99:
    status_color = "#28a745"  # Green
    status_emoji = "✅"
elif alignment_pct >= 0.95:
    status_color = "#ffc107"  # Yellow
    status_emoji = "⚠️"
else:
    status_color = "#dc3545"  # Red
    status_emoji = "❌"

print(f"📈 Overall Alignment: {alignment_pct:.1%}")
print(f"🎯 Health Status: {kpis['[Health_Status]']}")

## Step 5: Build HTML Email

In [ ]:
# Power BI report URL (replace with your actual report URL)
pbi_report_url = "https://app.powerbi.com/groups/YOUR_WORKSPACE_ID/reports/YOUR_REPORT_ID"

# HTML email template
email_body = f"""
<html>
<head>
    <style>
        body {{ font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; }}
        .container {{ max-width: 800px; margin: 0 auto; }}
        .header {{ background-color: #2E4057; color: white; padding: 20px; border-radius: 5px 5px 0 0; }}
        .status-banner {{ background-color: {status_color}; color: white; padding: 15px; margin: 20px 0; border-radius: 5px; }}
        table {{ width: 100%; border-collapse: collapse; margin: 20px 0; }}
        th {{ background-color: #2E4057; color: white; padding: 12px; text-align: left; }}
        td {{ padding: 12px; border-bottom: 1px solid #ddd; }}
        tr:nth-child(even) {{ background-color: #f8f9fa; }}
        .diff-positive {{ color: #28a745; font-weight: bold; }}
        .diff-negative {{ color: #dc3545; font-weight: bold; }}
        .diff-zero {{ color: #6c757d; }}
        .cta {{ background-color: #007bff; color: white; padding: 15px 30px; text-decoration: none; border-radius: 5px; display: inline-block; margin: 20px 0; }}
        .footer {{ color: #6c757d; font-size: 12px; margin-top: 30px; padding-top: 20px; border-top: 1px solid #ddd; }}
    </style>
</head>
<body>
    <div class="container">
        <!-- Header -->
        <div class="header">
            <h2>📊 Stock Validation Report - BIPROD vs Azure</h2>
            <p style="margin: 5px 0;">Report Date: <strong>{report_date}</strong></p>
        </div>
        
        <!-- Status Banner -->
        <div class="status-banner">
            <h3 style="margin: 0;">{status_emoji} {kpis['[Health_Status]']}</h3>
            <p style="margin: 5px 0 0 0;">Overall Alignment: <strong>{alignment_pct:.1%}</strong></p>
        </div>
        
        <!-- Overall Totals Table -->
        <h3 style="color: #2E4057; border-bottom: 2px solid #2E4057; padding-bottom: 10px;">Overall Totals</h3>
        <table>
            <tr>
                <th>Metric</th>
                <th style="text-align: right;">Azure</th>
                <th style="text-align: right;">BIPROD</th>
                <th style="text-align: right;">Difference</th>
                <th style="text-align: right;">% Diff</th>
            </tr>
            <tr>
                <td><strong>Records</strong></td>
                <td style="text-align: right;">{kpis['[Azure_Records]']:,.0f}</td>
                <td style="text-align: right;">{kpis['[BIPROD_Records]']:,.0f}</td>
                <td style="text-align: right;" class="{'diff-positive' if kpis['[Records_Diff]'] > 0 else 'diff-negative' if kpis['[Records_Diff]'] < 0 else 'diff-zero'}">
                    {kpis['[Records_Diff]']:+,.0f}
                </td>
                <td style="text-align: right;">—</td>
            </tr>
            <tr>
                <td><strong>Unique Barcodes</strong></td>
                <td style="text-align: right;">{kpis['[Azure_Unique_Barcodes]']:,.0f}</td>
                <td style="text-align: right;">{kpis['[BIPROD_Unique_Barcodes]']:,.0f}</td>
                <td style="text-align: right;" class="{'diff-positive' if kpis['[Barcodes_Diff]'] > 0 else 'diff-negative' if kpis['[Barcodes_Diff]'] < 0 else 'diff-zero'}">
                    {kpis['[Barcodes_Diff]']:+,.0f}
                </td>
                <td style="text-align: right;">—</td>
            </tr>
            <tr>
                <td><strong>Total QTY</strong></td>
                <td style="text-align: right;">{kpis['[Azure_Total_QTY]']:,.0f}</td>
                <td style="text-align: right;">{kpis['[BIPROD_Total_QTY]']:,.0f}</td>
                <td style="text-align: right;" class="{'diff-positive' if kpis['[Total_QTY_Diff_Correct]'] > 0 else 'diff-negative' if kpis['[Total_QTY_Diff_Correct]'] < 0 else 'diff-zero'}">
                    {kpis['[Total_QTY_Diff_Correct]']:+,.0f}
                </td>
                <td style="text-align: right;">{kpis['[QTY_Diff_Pct]']:.2%}</td>
            </tr>
            <tr>
                <td><strong>Total COGS</strong></td>
                <td style="text-align: right;">€{kpis['[Azure_Total_COGS]']:,.2f}</td>
                <td style="text-align: right;">€{kpis['[BIPROD_Total_COGS]']:,.2f}</td>
                <td style="text-align: right;" class="{'diff-positive' if kpis['[Total_COGS_Diff_Correct]'] > 0 else 'diff-negative' if kpis['[Total_COGS_Diff_Correct]'] < 0 else 'diff-zero'}">
                    €{kpis['[Total_COGS_Diff_Correct]']:+,.2f}
                </td>
                <td style="text-align: right;">{kpis['[COGS_Diff_Pct]']:.2%}</td>
            </tr>
        </table>
        
        <!-- Match Status Breakdown -->
        <h3 style="color: #2E4057; border-bottom: 2px solid #2E4057; padding-bottom: 10px;">Match Status Breakdown</h3>
        <table>
            <tr>
                <th>Status</th>
                <th style="text-align: right;">Count</th>
                <th style="text-align: right;">Percentage</th>
            </tr>
            <tr>
                <td>✅ <strong>Perfect Match</strong></td>
                <td style="text-align: right;">{kpis['[Match_Count]']:,.0f}</td>
                <td style="text-align: right;">{alignment_pct:.1%}</td>
            </tr>
            <tr>
                <td>❌ <strong>Mismatch</strong></td>
                <td style="text-align: right;">{kpis['[Mismatch_Count]']:,.0f}</td>
                <td style="text-align: right;">{(kpis['[Mismatch_Count]'] / (kpis['[Azure_Records]'] + kpis['[BIPROD_Records]'] - kpis['[Match_Count]'])):.1%}</td>
            </tr>
            <tr>
                <td>🔵 <strong>Azure Only</strong></td>
                <td style="text-align: right;">{kpis['[Azure_Only_Count]']:,.0f}</td>
                <td style="text-align: right;">—</td>
            </tr>
            <tr>
                <td>🟠 <strong>BIPROD Only</strong></td>
                <td style="text-align: right;">{kpis['[BIPROD_Only_Count]']:,.0f}</td>
                <td style="text-align: right;">—</td>
            </tr>
        </table>
        
        <!-- Call to Action -->
        <div style="text-align: center; margin: 30px 0;">
            <a href="{pbi_report_url}" class="cta">
                📈 View Full Interactive Dashboard →
            </a>
        </div>
        
        <!-- Footer -->
        <div class="footer">
            <p><strong>ℹ️ About This Report</strong></p>
            <p>This automated validation report compares stock data between BIPROD (legacy) and Azure (migration target) systems. 
               The report monitors data integrity during the migration process.</p>
            <p><strong>Next Report:</strong> {next_report}<br>
               <strong>Frequency:</strong> Twice weekly (Monday & Thursday, 8:00 AM)</p>
            <p style="margin-top: 15px;"><em>This is an automated report generated by Microsoft Fabric. Please do not reply to this email.</em></p>
        </div>
    </div>
</body>
</html>
"""

print("✅ Email HTML generated successfully")

## Step 6: Send Email via Microsoft Graph API

In [ ]:
# Email configuration
RECIPIENT_EMAIL = "manager@company.com"  # Replace with actual recipient
CC_EMAIL = "team@company.com"            # Optional CC

# Get authentication token
try:
    token = mssparkutils.credentials.getToken('https://graph.microsoft.com')
    print("🔐 Authentication token obtained")
except Exception as e:
    print(f"❌ Authentication failed: {str(e)}")
    raise

# Prepare email payload
email_payload = {
    "message": {
        "subject": f"Stock Validation Report - {report_date}",
        "body": {
            "contentType": "HTML",
            "content": email_body
        },
        "toRecipients": [
            {"emailAddress": {"address": RECIPIENT_EMAIL}}
        ],
        "ccRecipients": [
            {"emailAddress": {"address": CC_EMAIL}}
        ],
        "importance": "normal"
    },
    "saveToSentItems": True
}

# Send email
try:
    response = requests.post(
        "https://graph.microsoft.com/v1.0/me/sendMail",
        headers={
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json"
        },
        json=email_payload
    )
    
    if response.status_code == 202:
        print(f"✅ Email sent successfully at {datetime.now().strftime('%H:%M:%S')}")
        print(f"📧 Recipient: {RECIPIENT_EMAIL}")
        print(f"📋 Subject: Stock Validation Report - {report_date}")
    else:
        print(f"❌ Email failed with status code: {response.status_code}")
        print(f"Response: {response.text}")
        
except Exception as e:
    print(f"❌ Error sending email: {str(e)}")
    raise

## Step 7: Execution Summary

In [ ]:
# Print execution summary
print("\n" + "="*60)
print("📊 EXECUTION SUMMARY")
print("="*60)
print(f"✅ Report Date: {report_date}")
print(f"✅ Alignment Status: {kpis['[Health_Status]']}")
print(f"✅ Overall Alignment: {alignment_pct:.1%}")
print(f"✅ Total Records Compared: {kpis['[Azure_Records]'] + kpis['[BIPROD_Records]']:,.0f}")
print(f"✅ Email Sent To: {RECIPIENT_EMAIL}")
print(f"✅ Next Scheduled Run: {next_report}")
print("="*60)
print("\n🎉 Notebook execution completed successfully!")

---

## Scheduling Instructions

To schedule this notebook in Microsoft Fabric:

1. **Save the notebook** in your Fabric workspace
2. **Click on the Schedule button** in the notebook toolbar
3. **Configure the schedule:**
   - Frequency: Weekly
   - Days: Monday, Thursday
   - Time: 08:00 AM (UTC)
   - Timezone: Select your local timezone
4. **Enable the schedule**
5. **(Optional) Set up failure notifications** to alert you if the notebook fails

---

## Troubleshooting

**Common Issues:**

1. **Authentication Error:**
   - Ensure the notebook has permissions to send emails via Microsoft Graph
   - Check that the workspace service principal has proper permissions

2. **DAX Query Error:**
   - Verify that all DAX measure names match exactly (case-sensitive)
   - Ensure the dataset is refreshed and accessible

3. **Email Not Sending:**
   - Verify recipient email addresses
   - Check Microsoft Graph API permissions
   - Ensure network connectivity

---

**End of Notebook**